In [5]:
import os
from urllib import request
from bs4 import BeautifulSoup 
import time
#import zipfile
import gzip
import numpy as np
import pandas
from pprint import pprint
#import io
data_dir = '/home/idies/workspace/21cc/Data/'
jobs_dir = data_dir + 'LODES8/'
baltimore_dir = '/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/code_guide_lookups/'

os.chdir(jobs_dir)
this_state = 'wy'

g = 0
print('Done!')

#os.getcwd()

Done!


In [6]:
s = time.time()
y = os.listdir(data_dir)
hasdir = [x for x in y if x=='LODES8']
if (len(hasdir) == 0):
    os.mkdir(jobs_dir)
if ('{0:}'.format(this_state) not in [x for x in os.listdir(jobs_dir) if x[-4:] != '.pdf']):
    os.mkdir(jobs_dir+'{0:}'.format(this_state))
e = time.time()
g = g + (e-s)
print('Done in {0:,.3f} seconds!'.format(e-s))
sorted(os.listdir(jobs_dir))

Done in 0.006 seconds!


['LODESTechDoc8.1.pdf',
 'ak',
 'al',
 'ar',
 'az',
 'ca',
 'co',
 'ct',
 'dc',
 'de',
 'fl',
 'ga',
 'hi',
 'ia',
 'id',
 'il',
 'in',
 'ks',
 'ky',
 'la',
 'ma',
 'md',
 'me',
 'mi',
 'mn',
 'mo',
 'ms',
 'mt',
 'nc',
 'nd',
 'ne',
 'nh',
 'nj',
 'nm',
 'ny',
 'oh',
 'ok',
 'or',
 'pa',
 'ri',
 'sc',
 'sd',
 'tn',
 'tx',
 'ut',
 'va',
 'vt',
 'wa',
 'wi',
 'wv',
 'wy']

# Get LODES data from census.gov

In [7]:
s = time.time()

theurl = 'https://lehd.ces.census.gov/data/lodes/LODES8/'
#print(theurl)
#print('getting documentation as unopenable PDF...')
indexpage = request.urlopen(theurl)
soup = BeautifulSoup(indexpage, 'html.parser')
indexpage.close()

bigtable = soup.find('table')
latest_pdf_file_name = [x for x in bigtable.findAll('tr') if '.pdf' in x.get_text()][-1].find('a').get('href')
latest_pdf_url = theurl + latest_pdf_file_name
pdfobj = request.urlopen(latest_pdf_url).read()

with open(jobs_dir+latest_pdf_file_name, 'wb') as f:
    f.write(pdfobj)

# #print('getting state data...')
# allstaterows = bigtable.findAll('tr')

# for i in range(0, len(allstaterows)):
#     if (allstaterows[i].get_text()[:allstaterows[i].get_text().find('/')] == this_state):
#         print(allstaterows[i].get_text())
#state_url = 'https://lehd.ces.census.gov/data/lodes/LODES7/{0:}/'.format(this_state)
#print(state_url)

for thisdirtype in ['od', 'rac', 'wac']:
    if (thisdirtype not in os.listdir(jobs_dir+this_state)):
        print('creating directory structure...')
        os.mkdir(jobs_dir+this_state+'/'+thisdirtype)
#allstatecells = allstaterows[i].findAll('td')
version_text_file = request.urlopen('{0:}{1:}/version.txt'.format(theurl, this_state)).read()
print('{0:}{1:}'.format(theurl,this_state))

print('\tgetting version and crosswalk...')

crosswalk_zipfile_name = '{0:}_xwalk.csv.gz'.format(this_state)
crosswalk_zipfile = request.urlopen('{0:}{1:}/{2:}'.format(theurl,this_state,crosswalk_zipfile_name)).read()

with open('{0:}{1:}/{2}'.format(jobs_dir, this_state, crosswalk_zipfile_name), 'wb') as f:
    f.write(crosswalk_zipfile)

csvfilename = '{0:}{1:}/{2}'.format(jobs_dir, this_state, crosswalk_zipfile_name)[:-3]
with gzip.open('{0:}{1:}/{2}'.format(jobs_dir, this_state, crosswalk_zipfile_name), 'rb') as f:
    file_content = f.read()
with open(csvfilename, 'wb') as f:
    f.write(file_content)
os.remove('{0:}{1:}/{2}'.format(jobs_dir, this_state, crosswalk_zipfile_name))

print('\tgetting origin-destination data...')
odurl = '{0:}{1:}/od/'.format(theurl, this_state)
odpage = request.urlopen(odurl)

odsoup = BeautifulSoup(odpage, 'html.parser')
odtable = odsoup.find('table')
odrows = odtable.findAll('tr')
od_zipfile_list = [x.find('a').get('href') for x in odrows[3:-1]]# if 'JT00' in x.find('a').get('href')]
cnt = 0
for this_file in od_zipfile_list:
    if (np.mod(cnt, 50) == 0):
        print('\t\tDownloading file {0:,.0f} of {1:,.0f}...'.format(cnt, len(od_zipfile_list)))
    with open(jobs_dir+this_state+'/od/'+this_file, 'wb') as f:
        f.write(request.urlopen(odurl+this_file).read())
    cnt += 1


print('\tgetting workplace area data...')
wacurl = '{0:}{1:}/wac/'.format(theurl, this_state)
wacpage = request.urlopen(wacurl).read()
wacsoup = BeautifulSoup(wacpage, 'html.parser')
wactable = wacsoup.find('table')

wacrows = wactable.findAll('tr')
wac_zipfile_list = [x.find('a').get('href') for x in wacrows[3:-1]]# if 'S000_JT00' in x.find('a').get('href')]

cnt = 0
for this_file in wac_zipfile_list:
    if (np.mod(cnt, 100) == 0):
        print('\t\tDownloading file {0:,.0f} of {1:,.0f}...'.format(cnt, len(wac_zipfile_list)))
    with open(jobs_dir+this_state+'/wac/'+this_file, 'wb') as f:
        f.write(request.urlopen(wacurl+this_file).read())
    cnt += 1

print('unzipping OD files...')
od_csvfile_list = [jobs_dir+this_state+'/od/'+x for x in os.listdir(jobs_dir+this_state+'/od/') if x[-3:] == '.gz']
cnt = 0
for this_file_name in od_csvfile_list:
    if (np.mod(cnt, 50) == 0):
        print('\tUnzipping file {0:,.0f} of {1:,.0f}...'.format(cnt, len(od_csvfile_list)))
    csvfilename = this_file_name[:-3]
    with gzip.open(this_file_name, 'rb') as f:
        file_content = f.read()
    with open(csvfilename, 'wb') as f:
        f.write(file_content)
    os.remove(this_file_name)
    cnt +=1 

wac_csvfile_list = [jobs_dir+this_state+'/wac/'+x for x in os.listdir(jobs_dir+this_state+'/wac/') if x[-3:] == '.gz']

print('unzipping WAC files...')
cnt = 0
for this_file_name in wac_csvfile_list:
    if (np.mod(cnt, 100) == 0):
        print('\tUnzipping file {0:,.0f} of {1:,.0f}...'.format(cnt, len(wac_csvfile_list)))
    csvfilename = this_file_name[:-3]
    with gzip.open(this_file_name, 'rb') as f:
        file_content = f.read()
    with open(csvfilename, 'wb') as f:
        f.write(file_content)
    os.remove(this_file_name)
    cnt +=1         
e = time.time()
g = g + (e-s)
print('Done in {0:,.0f} minutes {1:,.0f} seconds!'.format(np.floor((g)/60), np.floor((g)%60)))


creating directory structure...
creating directory structure...
creating directory structure...
https://lehd.ces.census.gov/data/lodes/LODES8/wy
	getting version and crosswalk...
	getting origin-destination data...
	getting workplace area data...
unzipping OD files...
	Unzipping file 0 of 208...
	Unzipping file 50 of 208...
	Unzipping file 100 of 208...
	Unzipping file 150 of 208...
	Unzipping file 200 of 208...
unzipping WAC files...
	Unzipping file 0 of 1,012...
	Unzipping file 100 of 1,012...
	Unzipping file 200 of 1,012...
	Unzipping file 300 of 1,012...
	Unzipping file 400 of 1,012...
	Unzipping file 500 of 1,012...
	Unzipping file 600 of 1,012...
	Unzipping file 700 of 1,012...
	Unzipping file 800 of 1,012...
	Unzipping file 900 of 1,012...
	Unzipping file 1,000 of 1,012...
Done in 5 minutes 32 seconds!


In [ ]:
# AK: Done in 0:03:50
# AL: Done in 0:15:39
# AR: Done in 0:09:10
# AZ: Done in 0:16:43
# CA: Done in 1:31:45
# CO: Done in 0:16:15
# CT: Done in 0:12:44
# DC: Done in 0:04:18
# DE: Done in 0:06:00
# FL: Done in 0:49:53
# GA: Done in 0:27:01
# HI: Done in 0:06:03
# IA: Done in 0:12:38
# ID: Done in 0:07:39
# IL: Done in 0:36:15
# IN: Done in 0:19:28
# KS: Done in 0:11:53
# KY: Done in 0:14:04
# LA: Done in 0:15:31
# MA: Done in 0:14:36
# MD: Done in 0:18:05
# ME: Done in 0:07:05
# MI: Done in 0:27:11
# MN: Done in 0:18:19
# MO: Done in 0:20:20
# MS: Done in 0:07:45
# MT: Done in 0:06:40
# NC: Done in 0:28:15
# ND: Done in 0:06:13
# NE: Done in 0:11:07
# NH: Done in 0:06:55
# NJ: Done in 0:26:25
# NM: Done in 0:09:12
# NY: Done in 0:51:37
# OH: Done in 0:34:52
# OK: Done in 0:13:49
# OR: Done in 0:15:01
# PA: Done in 0:39:05
# RI: Done in 0:06:45
# SC: Done in 0:16:42
# SD: Done in 0:06:49
# TN: Done in 0:27:11
# TX: Done in 1:07:49
# UT: Done in 0:11:28
# VA: Done in 0:25:18
# VT: Done in 0:05:50
# WA: Done in 0:21:42
# WI: Done in 0:19:52
# WV: Done in 0:07:58
# WY: Done in 0:05:32

### 